In [ ]:
## Курсовой проект для курса "Python для Data Science"

Материалы к проекту (файлы): train.csv test.csv

Задание: 
Используя данные из обучающего датасета (train.csv), построить модель для предсказания цен на недвижимость (квартиры). 
С помощью полученной модели, предсказать цены для квартир из тестового датасета (test.csv).

Целевая переменная: Price
Метрика качества: R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Требования к решению:
R2 > 0.6
Тетрадка Jupyter Notebook с кодом Вашего решения, названная по образцу {ФИО}_solution.ipynb, пример SShirkin_solution.ipynb
Файл CSV с прогнозами целевой переменной для тестового датасета, названный по образцу {ФИО}_predictions.csv, пример SShirkin_predictions.csv 
Файл должен содержать два поля: Id, Price и в файле должна быть 5001 строка (шапка + 5000 предсказаний).

Сроки сдачи: Cдать проект нужно в течение 72 часов после окончания последнего вебинара. Оценки работ, сданных до дедлайна, будут представлены в виде рейтинга, ранжированного по заданной метрике качества. Проекты, сданные после дедлайна или сданные повторно, не попадают в рейтинг, но можно будет узнать результат.

Рекомендации для файла с кодом (ipynb):
Файл должен содержать заголовки и комментарии (markdown)
Повторяющиеся операции лучше оформлять в виде функций
Не делать вывод большого количества строк таблиц (5-10 достаточно)
По возможности добавлять графики, описывающие данные (около 3-5)
Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
Весь проект должен быть в одном скрипте (файл ipynb).
Допускается применение библиотек Python и моделей машинного обучения, которые были в данном курсе.

Описание датасета: 
Id - идентификационный номер квартиры 

DistrictId - идентификационный номер района 

Rooms - количество комнат 

Square - площадь 

LifeSquare - жилая площадь 

KitchenSquare - площадь кухни 

Floor - этаж 

HouseFloor - количество этажей в доме 

HouseYear - год постройки дома 

Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности 

Social_1, Social_2, Social_3 - социальные показатели местности 

Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья 

Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров 

Price - цена квартиры

# РЕШЕНИЕ

Загрузим нужные библиотеки

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')
plt.rcParams.update({'font.size': 14})
%matplotlib inline

from scipy.stats import norm
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.manifold import TSNE
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier

Загрузим файлы к проекту: train.csv test.csv

train = pd.read_csv('/Users/ekaterina/Downloads/project_task/train.csv')
test = pd.read_csv('/Users/ekaterina/Downloads/project_task/test.csv')
print('Материалы к проекту загружены!\n')
print(f'Размер массива Train:\n{train.shape[0]}\tквартир and {train.shape[1]} признаков\n')
print(f'Размер массива Test:\n{test.shape[0]}\tквартир and {test.shape[1]} признаков')

Посмотрим первые 5 строк в датасетах:

train.head()

test.head()

Посмотрим количество заполненных значений и типы данных:

train.info(memory_usage='deep')

test.info(memory_usage='deep')

Оптимизируем использование памяти:

train['Rooms'] = train['Rooms'].astype('int16')
train['Floor'] = train['Floor'].astype('int16')
train['DistrictId'] = train['DistrictId'].astype('int32')
train['HouseFloor'] = train['HouseFloor'].astype('int32')
train['HouseYear'] = train['HouseYear'].astype('int32')
train['Square'] = train['Square'].astype('float32')
train['LifeSquare'] = train['LifeSquare'].astype('float32')
train['KitchenSquare'] = train['KitchenSquare'].astype('float32')
train['Price'] = train['Price'].astype('float32')
train['Healthcare_1'] = train['Healthcare_1'].astype('float32')
train['Helthcare_2'] = train['Helthcare_2'].astype('int32')
train['Social_1'] = train['Social_1'].astype('int32')
train['Social_2'] = train['Social_2'].astype('int32')
train['Social_3'] = train['Social_3'].astype('int32')
train['Shops_1'] = train['Shops_1'].astype('int32')
train['Ecology_1'] = train['Ecology_1'].astype('float32')

train.info(memory_usage='deep')

train.describe()

# Поработаем с данными:

0. Исключим признак id

train = train[train.columns[1:]]

1. HouseYear

train[train['HouseYear']>2020]

train.loc[train['HouseYear'] == 20052011, 'HouseYear'] = int((2005 + 2011) / 2)
train.loc[train['HouseYear'] == 4968, 'HouseYear'] = 1968

train['HouseYear'].unique()

2. Square

train.loc[(train['Square'] < 3)].transpose()

Удалим строки с квартирами площадью меньше 3 м2

train = train.drop(1608, axis=0)
train = train.drop(6392, axis=0)
train = train.drop(4853, axis=0)
train = train.drop(8283, axis=0)
train = train.drop(9294, axis=0)

train.loc[(train['Square'] < 3)]

Квартиры с площадью больше, чем 600м2 скорее всего опечатки, разделим на 10

train.loc[(train['Square'] > 600)]

train.loc[train['Square'] > 600, 'Square'] = train.loc[train['Square'] > 600, 'Square']/10
train.loc[train['LifeSquare'] > 600, 'LifeSquare'] = train.loc[train['LifeSquare'] > 600, 'LifeSquare']/10

train.loc[4262]

train.loc[6977]

Квартиры с площадью мельше, чем 6м2 скорее всего опечатки, умножим на 10

train.loc[(train['Square'] < 6)].transpose()

train.loc[train['Square']  < 6, 'Square'] = train.loc[train['Square'] < 6, 'Square']*10

3. LifeSquare

Заполним недостающие данные по жилой площади и показатели местности, связанные с охраной здоровья

train[train['LifeSquare'].isnull()]

train.loc[(train['LifeSquare'].isnull(), 'LifeSquare')] = train['Square']-10

train.loc[(train['LifeSquare'] - train['Square'] >= 3)]

train.loc[(train['Square'] < train['LifeSquare'], 'LifeSquare')] = train['Square']-10

train.loc[(train['Square'] < train['KitchenSquare'])]

train['KitchenSquare'].median()

train.loc[(train['Square'] < train['KitchenSquare']), 'KitchenSquare'] = train['KitchenSquare'].median()
train.loc[(train['KitchenSquare'] == 0), 'KitchenSquare'] = train['KitchenSquare'].median()

train.loc[(train['Healthcare_1'].isnull(), 'Healthcare_1')] = train['Healthcare_1'].median()

train[train['Healthcare_1'].isnull()]

4. Rooms & Floors

train.loc[((train['Rooms'] > 5)&(train['Square'] < 70))|(train['Rooms'] == 0)]

train.loc[(((train['Rooms'] > 5)&(train['Square'] < 70))|(train['Rooms'] == 0)), 'Rooms'] = train['Square']//20

train.loc[1454]

train.loc[(train['HouseFloor'] == 0)]

train['HouseFloor'].median()

Так как медиана меньше некоторых этажей, она не подходит для замены нулей

train.loc[(train['HouseFloor'] == 0), 'HouseFloor'] = train['Floor']

train.loc[((train['HouseFloor'] > 50))]

train.loc[((train['HouseFloor'] > 50)), 'HouseFloor'] = train['HouseFloor']//10

5. Заполним оставшиеся пропуски модой или медианой

#train['DistrictId'].median()

train['Ecology_1'].mode()

train['Social_1'].median()

train['Social_3'].median()

train['Healthcare_1'].median()

#train['Healthcare_2'].mode()

train['Shops_1'].median()

train.loc[train['Shops_1'] == 0].transpose()

train.loc[(train['DistrictId'] == 0), 'DistrictId'] = train['DistrictId'].median()

#train.loc[(train['Ecology_1'] == 0), 'Ecology_1'] = train['Ecology_1'].mode()

train.loc[(train['Social_1'] == 0), 'Social_1'] = train['Social_1'].median()

train.loc[(train['Social_3'] == 0), 'Social_3'] = train['Social_3'].median()

train.loc[(train['Healthcare_1'] == 0), 'Healthcare_1'] = train['Healthcare_1'].median()

#train.loc[(train['Helthcare_2'] == 0), 'Helthcare_2'] = train['Helthcare_2'].median()


train.loc[(train['Shops_1'] == 0), 'Shops_1'] = train['Shops_1'].median()

train['Shops_1'].unique()

6. Преобразуем категориальные признаки в бинарные:

    -получим доп.таблицы
    
    -присоединим их к датасету
    
    -удалим столбцы с нечисловыми значениями

train['Ecology_2'].unique()

train['Shops_2'].unique()

pd.get_dummies(train['Ecology_2'])[:2]
pd.get_dummies(train['Ecology_3'])[:2]
pd.get_dummies(train['Shops_2'])[:2]

train = pd.concat([train, pd.get_dummies(train['Ecology_2'])], axis = 1)
train = pd.concat([train, pd.get_dummies(train['Ecology_3'])], axis = 1)
train = pd.concat([train, pd.get_dummies(train['Shops_2'])], axis = 1)

train = train.drop(['Ecology_2'], axis = 1)
train = train.drop(['Ecology_3'], axis = 1)
train = train.drop(['Shops_2'], axis = 1)

Посмотрим на обработанные данные

train.info(memory_usage='deep')

train.describe().transpose()

Сохраним обработанные данные в файл

train.to_csv('/Users/ekaterina/Downloads/project_task/train_prepared.csv', index=False, encoding='utf-8')

# Визуализируем подготовленные данные

Посмотрим корреляцию данных:

corrmat = train.loc[:, (train.columns != 'A')&(train.columns != 'B')].corr()
plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=0.5, square=True)

Видим наибольшую корреляцию между ценой и площадью квартиры, количеством комнат, а также жилой площадью. 

Среднее влияние на цену оказывают параметры: 
    район,  площадь кухни, социальные показатели местности, показатели местности, связанные с охраной здоровья, с наличием магазинов, торговых центров

Малое влияние на цену оказывают параметры:
    этаж, количество этажей в доме, год строительства
    
Экологические показатели местности практически не влияют на цену, поэтому при прогнозе их можно не учитывать.

Похожую картину мы видим на следующей диаграмме:

corrmat = train.loc[:, train.columns != 'Price'].corrwith(
    train['Price']).abs().sort_values(ascending=True)[0:]
plt.bar(corrmat.values, corrmat.index)
plt.title('Correlation to Price')
plt.xticks()
plt.show()

df = pd.read_csv('/Users/ekaterina/Downloads/project_task/train_prepared.csv')
df.head()

df.shape

df.dtypes

df.info(memory_usage='deep')

Отбор примеров

df['Price'].value_counts().head(10)

plt.figure(figsize = (16, 8))

df['Price'].hist(bins=30)
plt.ylabel('count')
plt.xlabel('Price')

plt.title('Distribution of Price')
plt.show()

Отбор признаков

df.columns.tolist()

feature_names = ['DistrictId', 'Rooms','Square','LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 'Social_1',
 'Social_2',
 'Social_3',
 'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 'A','B','A.1','B.1','A.2','B.2']

target_name = 'Price'

df = df[feature_names + [target_name]]
df.head()

Стандартизация признаков

df.mean()

feature_names_for_stand = df[feature_names].select_dtypes(include='float64').columns.tolist()
feature_names_for_stand

# MinMaxScaler
# fit - преобразование данных / обучение модели
# predict / transform

scaler = StandardScaler()
stand_features = scaler.fit_transform(df[feature_names_for_stand])

df[feature_names_for_stand] = pd.DataFrame(stand_features, columns=feature_names_for_stand)

df.head()

df.mean()

df.std()

Обучающий датасет готов

Разобьем данные на тренировочный и тестовый датасеты:

X = df[feature_names]
y = df[target_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=100)

Построение базовых моделей и выбор лучшей

Загрузим метрики

MAE - Mean Absolute Error 

MSE - Mean Squared Error

RMSE - Root Mean Squared Error

R2 - coefficient of determination

from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
from math import sqrt

def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 3)) + "\n" +
          "MAE:\t" + str(round(mae(true_values, pred_values), 3)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 3)) + "\n" +
          "RMSE:\t" + str(round(sqrt(mse(true_values, pred_values)), 3)))
    
    plt.figure(figsize=(10,10))
    
    sns.scatterplot(x=pred_values, y=true_values)
    
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('True vs Predicted values')
    plt.show()
    return None

Построим модель линейной регрессии:

lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

y_train_preds = lr_model.predict(X_train)
evaluate_preds(y_train, y_train_preds)

y_test_preds = lr_model.predict(X_test)
evaluate_preds(y_test, y_test_preds)

Random Forest Regressor

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

y_train_preds = rf_model.predict(X_train)
evaluate_preds(y_train, y_train_preds)

y_test_preds = rf_model.predict(X_test)
evaluate_preds(y_test, y_test_preds)

Отложенная выборка

def evaluate_preds_sec(train_true_values, train_pred_values, test_true_values, test_pred_values):
    """
    # дописать документацию0
    """
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

y_train_preds = rf_model.predict(X_train.fillna(-9999))
y_test_preds = rf_model.predict(X_test)

evaluate_preds_sec(y_train, y_train_preds, y_train, y_train_preds)

**Перекрёстная проверка**

cv_score = cross_val_score(rf_model, X.fillna(-9999), y, scoring='r2', cv=KFold(n_splits=5, shuffle=True, random_state=21))
cv_score

cv_score.mean(), cv_score.std()


cv_score.mean() - cv_score.std(), cv_score.mean() + cv_score.std()

**Важность признаков**

feature_importances = pd.DataFrame(zip(X_train.columns, rf_model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

Gradient Boosting Regressor

gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

y_train_preds = gb_model.predict(X_train)
evaluate_preds(y_train, y_train_preds)

y_test_preds = gb_model.predict(X_test)
evaluate_preds(y_test, y_test_preds)

Настройка и оценка финальной модели

Подбор гиперпараметров

0.6*X_train.shape[1]

gb_model = GradientBoostingRegressor(random_state=42)
gb_model

params = {'n_estimators':[50, 100, 200, 400], 
          'max_depth':[3, 5, 7, 10]}

gs = GridSearchCV(gb_model, params, scoring='r2', cv=KFold(n_splits=3, random_state=42, shuffle=True), n_jobs=-1)
gs.fit(X, y)

gs.best_params_

gs.best_score_

`GridSearchCV` - это классификатор, который строится на основе модели `estimator`, пробегая все комбинации значений из `param_grid`. Для каждой комбинации параметров по кросс-валидации на указанном количестве _фолдов_ считается метрика, указанная в `scoring`. Наконец, выбирается та комбинация параметров, при которой выбранная метрика оказалась максимальной, и дальше для предсказания используется именно этот набор параметров.

Обучение и оценка модели

final_model = GradientBoostingRegressor(n_estimators=400, max_depth=7, random_state=21)
final_model.fit(X_train, y_train)

y_train_preds = final_model.predict(X_train)
evaluate_preds(y_train, y_train_preds)

y_test_preds = final_model.predict(X_test)
evaluate_preds(y_test, y_test_preds)

Важность признаков

feature_importances = pd.DataFrame(zip(X_train.columns, final_model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

Сохранение модели

**Scaler**

df.to_csv('/Users/ekaterina/Downloads/project_task/scaler.pkl')

with open('/Users/ekaterina/Downloads/project_task/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('/Users/ekaterina/Downloads/project_task/scaler.pkl', "rb") as file:
    new_scaler = pickle.load(file)

**Model**

df.to_csv('/Users/ekaterina/Downloads/project_task/model.pkl')

with open('/Users/ekaterina/Downloads/project_task/model.pkl', 'wb') as file:
    pickle.dump(final_model, file)

Результат:



Выводы:





Выгрузка данных:

test[['Id', 'Price']].to_csv('/Users/ekaterina/Downloads/project_task/Macerszmidt_predictions.csv', index=None)



## Алгоритм t-SNE

Алгоритм _t-SNE_ (_t-distributed Stochastic Neighbor Embedding_ или _Стохастическое вложение соседей с t-распределением_) позволяет понижать размерность данных до двух или трёх измерений, что позволяет визуализировать данные на двумерных и трёхмерных графиках. Изучая графики, можно, например, понять, на сколько кластеров адекватно разбивать данные, а также оценить уже выполненное разбиение на кластеры.

Итак, перейдём к использованию t-SNE. 
Зададим параметр `n_components=2`, чтобы получить данные с двумя признаками. 
Параметр `learning_rate` влияет на то, как плотно будут располагаться точки. 
Рекомендуется задавать его в диапазоне от 10 до 1000.


#scaler = StandardScaler()
stand_features = scaler.fit_transform(df[feature_names_for_stand])

df[feature_names_for_stand] = pd.DataFrame(stand_features, columns=feature_names_for_stand)

tsne = TSNE(n_components=2, learning_rate=150, random_state=100)

X_train_tsne = tsne.fit_transform(df[feature_names_for_stand])

print('До:\t{}'.format(df[feature_names_for_stand].shape))
print('После:\t{}'.format(X_train_tsne.shape))

Мы видим, что число признаков уменьшилось с 10 до 2. Теперь можно визуализировать наши данные на плоскости.

plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1])

plt.show()

По графику видно, что данные можно разбить как минимум на 2 кластера. Попробуем сделать это с помощью уже известного нам метода K-means и ещё раз построим график, но уже с полученными метками кластеров.



kmeans = KMeans(n_clusters=2)

labels_train = kmeans.fit_predict(df[feature_names_for_stand])






plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=labels_train)

plt.show()

Как мы видим, при кластеризации мы получили практически такое же разбиение, как то, которое можно было наблюдать в результате t-SNE.

Для тестовой выборки получить аналогичный график нам не удастся, поскольку у алгоритма t-SNE нет метода `.transform`. Однако, с помощью алгоритма K-means мы можем получить метки кластеров для тестовой выборки с помощью метода `.predict` и использовать их:

Давайте посмотрим на объекты из разных кластеров, чтобы попытаться понять, почему алгоритм t-SNE выделил две группы. Например, посмотрим на среднюю цену недвижимости во всей выборке и в отдельных кластерах.



labels_test = kmeans.predict(stand_features)

y_train.mean()

y_train[labels_train == 1].mean()

plt.hist(y_train[labels_train == 0], bins=20, density=True, alpha=0.5)
plt.hist(y_train[labels_train == 1], bins=20, density=True, alpha=0.5)



labels_test = kmeans.predict(X_test_scaled)


y_train.mean()

y_train[labels_train == 0].mean()

y_train[labels_train == 1].mean()



вот здесь выдает ошибку и не дает дальше идти